## Preprocessing

In [72]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [73]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_low = application_df.drop(columns=['EIN', 'NAME'])
application_df_low

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [74]:
# Determine the number of unique values in each column.
application_df_nu = application_df_low.nunique()
application_df_nu

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [75]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df_low['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [76]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T2", "T12", "T9", "T13", "T14", "T15", "T17", "T25", "T29", "T10"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df_low['APPLICATION_TYPE'] = application_df_low['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df_low['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [77]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
nu_counts_class = application_df_low['CLASSIFICATION'].value_counts()
nu_counts_class 

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C6100        1
C1820        1
C1900        1
C2150        1
Name: count, Length: 71, dtype: int64

In [78]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for CLASSIFICATION, count in nu_counts_class.items():
        if count > 1:
                print(f"{CLASSIFICATION}: {count}")

C1000: 17326
C2000: 6074
C1200: 4837
C3000: 1918
C2100: 1883
C7000: 777
C1700: 287
C4000: 194
C5000: 116
C1270: 114
C2700: 104
C2800: 95
C7100: 75
C1300: 58
C1280: 50
C1230: 36
C1400: 34
C7200: 32
C2300: 32
C1240: 30
C8000: 20
C7120: 18
C1500: 16
C6000: 15
C1800: 15
C1250: 14
C8200: 11
C1238: 10
C1278: 10
C1237: 9
C1235: 9
C7210: 7
C1720: 6
C2400: 6
C4100: 6
C1257: 5
C1600: 5
C1260: 3
C2710: 3
C0: 3
C3200: 2
C1256: 2
C1246: 2
C1234: 2
C1267: 2


In [79]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
nu_counts_class_series = pd.Series(nu_counts_class)
classifications_to_replace = nu_counts_class_series[nu_counts_class_series < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
     application_df_low['CLASSIFICATION'] = application_df_low['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df_low['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [80]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(
    application_df_low, 
    columns=[
        "APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", 
        "USE_CASE", "ORGANIZATION", "INCOME_AMT", 
        "SPECIAL_CONSIDERATIONS"
    ]
)
print(df_dummies)

       STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                    True   
1           1    108590              1                   False   
2           1      5000              0                   False   
3           1      6692              1                   False   
4           1    142590              1                   False   
...       ...       ...            ...                     ...   
34294       1      5000              0                   False   
34295       1      5000              0                   False   
34296       1      5000              0                   False   
34297       1      5000              1                   False   
34298       1  36500179              0                   False   

       APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     False                False                False   
1                     False                 True             

In [81]:
# Split our preprocessed data into our features and target arrays
y = df_dummies["IS_SUCCESSFUL"]
X = df_dummies.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [82]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [83]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=42, name="input"))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu", name="hidden_1"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu", name="hidden_2"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", name="hidden_3"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid", name="output"))

# Check the structure of the model
nn.summary()

c:\Users\Sibal_1w46aws\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (Dense)                   │ (None, 80)             │         3,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_1 (Dense)                │ (None, 50)             │         4,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_2 (Dense)                │ (None, 40)             │         2,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_3 (Dense)                │ (None, 30)             │         1,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,791 (42.15 KB)

 Trainable params: 10,791 (42.15 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", 
    optimizer="adam",#tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), 
    metrics=["accuracy"]
)

# Define a callback to save weight every 5 epochs
checkmodel_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoints/model_weights_epoch_{epoch:02d}.weights.h5", 
    save_weights_only=True, 
    save_freq="epoch"
)

In [ ]:
# Train the model
fit_model = nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=100#, 
    # callbacks=[checkmodel_callback]
)

Epoch 1/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7044 - loss: 0.5919
Epoch 2/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7303 - loss: 0.5559
Epoch 3/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7321 - loss: 0.5511
Epoch 4/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7310 - loss: 0.5518
Epoch 5/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7330 - loss: 0.5473
Epoch 6/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7298 - loss: 0.5512
Epoch 7/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7359 - loss: 0.5462
Epoch 8/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7305 - loss: 0.5490
Epoch 9/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7310 - loss: 0.5512
Epoch 10/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7336 - loss: 0.5450
Epoch 11/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7373 - loss: 0.5413
Epoch 12/50
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

In [86]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 4ms/step - accuracy: 0.7289 - loss: 0.5583
Loss: 0.5583282709121704, Accuracy: 0.728863000869751


In [158]:
# Export our model to HDF5 file
nn.save("models/first_model.h5")